In [5]:
%reset
import matplotlib.pyplot as plt
import rasterio 
from rasterio.plot import plotting_extent, show
from rasterio.merge import merge
import geopandas as gpd
from osgeo import gdal
import numpy as np
from skimage import io


def data_fusion(ortho_nir, ortho_rgb, ortho_mspec, outputpath):
    
    ortho_NIR = ortho_nir
    ortho_rgb = ortho_rgb
    ortho_mspec = ortho_mspec
    
    g = gdal.Open(ortho_NIR)
    c = gdal.Open(ortho_mspec)
    x = gdal.Open(ortho_rgb)
    
    dsReprj_mspec= gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_mspec_5.tif", c, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    dsReprj_nir = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_nir_5.tif", g, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    dsReprj_rgb = gdal.Warp("D:/cbr/VP2/VP2_SoilMapping/CodeOutput/demRepr_rgb_5.tif", x, xRes = x.GetGeoTransform()[1], yRes= x.GetGeoTransform()[5])
    
    stack = []
   
    rgb_red = dsReprj_rgb.GetRasterBand(1).ReadAsArray().astype(np.uint8)
    rgb_green = dsReprj_rgb.GetRasterBand(2).ReadAsArray().astype(np.uint8)
    rgb_blue =  dsReprj_rgb.GetRasterBand(3).ReadAsArray().astype(np.uint8)
    stack.append(rgb_red)
    stack.append(rgb_green)
    stack.append(rgb_blue)
    
    mspec_rededge = dsReprj_mspec.GetRasterBand(3).ReadAsArray().astype(np.uint8) 
    stack.append(mspec_rededge)
   
    b3 = dsReprj_nir.GetRasterBand(2).ReadAsArray().astype(np.uint8)
    b4 = dsReprj_nir.GetRasterBand(3).ReadAsArray().astype(np.uint8)
    nir_ndvi = (b4 - b3)/(b4 + b3)
    nir_nir = dsReprj_nir.GetRasterBand(3).ReadAsArray().astype(np.uint8)
    stack.append(nir_nir)
    stack.append(nir_ndvi)
 
    print(len(stack),len(stack[0]))
    stack_d = np.empty((0,6), dtype= object)
    print(type(stack))
    print(stack_d)
    stack_d = np.append(stack_d, np.array([rgb_red, rgb_green,rgb_blue, mspec_rededge, nir_nir, nir_ndvi]), axis=0)
    #stack_data = np.asarray(stack, dtype=object)
    #stack_data = np.array(stack, dtype=object)
   
    #driver = gdal.GetDriverByName ( "GTiff" )
    #DataSet = driver.Create(outputpath, stack_data[4].shape[1], stack_data[4].shape[0], len(stack_data), gdal.GDT_Float32)
    #DataSet.SetGeoTransform(x.GetGeoTransform())
    #DataSet.SetProjection(x.GetProjection())
    #print('---------------------')
    #for i, image in enumerate(stack_data, 1):
    #    DataSet.GetRasterBand(i).WriteArray(image)
    #    DataSet.GetRasterBand(i).SetNoDataValue(-9999.0)
    #DataSet.FlushCache()

In [6]:
#ortho_NIR = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/Wetzwil_gross_nir.tif"
#ortho_rgb = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/Wetzwil_gross.tif"
#ortho_mspec = "D:/cbr/VP2/VP2_SoilMapping/Processed/Wetzwil_Ortho/Wetzwil_gross_mspec.tif"
#outputpath = "D:/cbr/VP2/VP2_SoilMapping/CodeOutput/channelFusion_Wetzwil.tif"

ortho_NIR = "D:/cbr/VP2/VP2_SoilMapping/Processed/orthophoto_Wetzwil_NIR.tif"
ortho_rgb = "D:/cbr/VP2/VP2_SoilMapping/Processed/orthophoto_Wetzwil_rgb.tif"
ortho_mspec = "D:/cbr/VP2/VP2_SoilMapping/Processed/orthophoto_Wetzwil_mspec.tif"
outputpath = "D:/cbr/VP2/VP2_SoilMapping/CodeOutput/channelFusion_Ortho_Wetzwil.tif"

data_fusion(ortho_NIR, ortho_rgb, ortho_mspec, outputpath)

C:\Users\ST1154~1\AppData\Local\Temp/ipykernel_11836/3394642761.py:40: RuntimeWarning: divide by zero encountered in true_divide
  nir_ndvi = (b4 - b3)/(b4 + b3)
C:\Users\ST1154~1\AppData\Local\Temp/ipykernel_11836/3394642761.py:40: RuntimeWarning: invalid value encountered in true_divide
  nir_ndvi = (b4 - b3)/(b4 + b3)


6 28102
<class 'list'>
[]


C:\Users\ST1154~1\AppData\Local\Temp/ipykernel_11836/3394642761.py:49: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  stack_d = np.append(stack_d, np.array([rgb_red, rgb_green,rgb_blue, mspec_rededge, nir_nir, nir_ndvi]), axis=0)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)